<a href="https://colab.research.google.com/github/AnIsAsPe/LDATopicModeling_pyspark/blob/main/LDA_con_sklearn20_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyLDAvis  #biblioteca que extrae información de un modelo LDA para obtener una visualización interactiva

In [ ]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer

import pyLDAvis
import matplotlib.pyplot as plt 
import seaborn as sns  

nltk.download('stopwords')
nltk.download('wordnet')  #WordNetLemmatizer
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

En colab, al instalar la librería pyLDAvis causa un conflicto con Pandas. Si esto se refleja al correr la siguiente linea es necesario reiniciar el entorno de ejecución.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datos/abcnews-date-text.csv', parse_dates=['publish_date'] )
print(df.shape)
df.head()

(1226258, 2)


,publish_date,headline_text
0,2003-02-19,aba decides against community broadcasting lic...
1,2003-02-19,act fire witnesses must be aware of defamation
2,2003-02-19,a g calls for infrastructure protection summit
3,2003-02-19,air nz staff in aust strike for pay rise
4,2003-02-19,air nz strike to affect australian travellers


In [ ]:
df.publish_date.min(), df.publish_date.max()

(Timestamp('2003-02-19 00:00:00'), Timestamp('2020-12-31 00:00:00'))

In [ ]:
# promedio de noticias por día
len(df)/(df.publish_date.max()-df.publish_date.min()).days


187.93226053639847

In [ ]:
df.sample(25)

,headline_text
924273,shark drum lines set off warnbro day 3
808096,nrl wrap sunday august 4
56451,police seek three missing in melbourne
714173,perfect weather for fraser is burn offs
795246,miners still optimisitc for oakajee
697070,researchers develop new dengue virus identific...
97100,commitment shown to lake eyre basin sustainabi...
830996,ocean of cricket details rise of sport across ...
252602,oxfam shop auctions break in weapon
954866,suspected rohingya boats evade malaysian rescu...


In [ ]:
def preprocesar(texto):
  #convierte a minúsculas
  texto = texto.lower()

  #elimina stopwords
  stop = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
  texto = stop.sub('', texto) 

  #lematizar y quedarnos con palabras que tengan más de tres caracteres
  lemmatizer = WordNetLemmatizer()
  texto = texto.split()
  texto = ' '.join([lemmatizer.lemmatize(i) for i in texto if len(i)>3])
  
  return(texto)

In [ ]:
%%time
df['headline_pp']=df['headline_text'].apply(preprocesar)
df.head()

CPU times: user 3min 11s, sys: 15.2 s, total: 3min 26s
Wall time: 3min 26s


In [ ]:
#vectorizer = TfidfVectorizer()           # 92403 tamaño de vocabulario
vectorizer = TfidfVectorizer(min_df=3)    #42695
vectorizer = TfidfVectorizer(min_df=20, ngram_range=(1,2))
BOW = vectorizer.fit_transform(df['headline_pp'])
BOW.shape

(1226258, 36660)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=20,learning_method='online',random_state=42,max_iter=10) 

In [ ]:
%%time
lda_top=lda_model.fit_transform(BOW)

CPU times: user 1h 12min 34s, sys: 23.2 s, total: 1h 12min 57s
Wall time: 1h 12min 48s


In [ ]:
import pickle
with open('/content/drive/MyDrive/Modelos/modelosLDA/LDA_sklearn/LDA20_10iter.pikle', 'wb') as f:
  pickle.dump(lda_top, f)

In [ ]:
import pickle
with open('/content/drive/MyDrive/Modelos/modelosLDA/LDA_sklearn/vectorizer.pikle', 'wb') as f:
  pickle.dump(lda_top, f)

In [ ]:
import pickle
with open('/content/drive/MyDrive/Modelos/modelosLDA/LDA_sklearn/BOW.pikle', 'wb') as f:
  pickle.dump(BOW, f)

In [ ]:
%%time
lda_model.perplexity(BOW, sub_sampling = False)

CPU times: user 1min 48s, sys: 887 ms, total: 1min 49s
Wall time: 1min 48s


83968.11437980646

In [ ]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda_model, BOW, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
12    -0.181567 -0.052764       1        1  5.621199
9      0.096530  0.076398       2        1  5.404073
3      0.113550 -0.055457       3        1  5.173005
10    -0.066508 -0.094020       4        1  5.132125
15     0.118551  0.168088       5        1  5.050068
2      0.111031  0.130990       6        1  5.049910
19    -0.107039 -0.168078       7        1  5.038129
16     0.158912 -0.154940       8        1  5.028129
0      0.114011  0.082870       9        1  5.017374
18    -0.174603  0.104250      10        1  4.961355
13     0.032126  0.079531      11        1  4.947165
14    -0.119641  0.148698      12        1  4.946355
7     -0.133201  0.007058      13        1  4.933431
8     -0.016131 -0.005134      14        1  4.924098
6      0.065865 -0.150169      15        1  4.910854
4     -0.125871  0.010796      16        1  4.904815
5      0.021464 -0.045856      17        1  4.803747
1      0.157828  0.012736      18        1  4.741924
17    -0.073024  0.110813      19        1  4.725842
11     0.007715 -0.205809      20        1  4.686401, topic_info=              Term         Freq        Total Category  logprob  loglift
2060     australia  7515.000000  7515.000000  Default  30.0000  30.0000
7467   coronavirus  6641.000000  6641.000000  Default  29.0000  29.0000
2166    australian  5839.000000  5839.000000  Default  28.0000  28.0000
35982        woman  4868.000000  4868.000000  Default  27.0000  27.0000
17174    interview  4743.000000  4743.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
25526        probe  1153.255197  1346.888065  Topic20  -4.7284   2.9053
3809         boost  1260.286736  1593.457671  Topic20  -4.6397   2.8259
24347         plan  1859.189769  3523.767421  Topic20  -4.2509   2.4211
16924      injured   827.436399  1256.478960  Topic20  -5.0604   2.6428
24622       police   871.334503  7387.554815  Topic20  -5.0087   0.9230

[714 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
113        3  0.998623     2014
116        2  0.999427     2015
118       11  0.998888     2016
122       17  0.998508     2018
126       11  0.998836     2020
...      ...       ...      ...
36485      4  0.225028     year
36485     17  0.774727     year
36562     13  0.999517    young
36585      7  0.998796    youth
36621      5  0.998929  zealand

[798 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 10, 4, 11, 16, 3, 20, 17, 1, 19, 14, 15, 8, 9, 7, 5, 6, 2, 18, 12])